In [2]:
import pandas as pd
import json

In [3]:
# Load the CSV data into a DataFrame
df = pd.read_csv("df_fuel_ckan.csv", parse_dates=["DATETIME"])
df.head()

,DATETIME,GAS,COAL,NUCLEAR,WIND,HYDRO,IMPORTS,BIOMASS,OTHER,SOLAR,...,IMPORTS_perc,BIOMASS_perc,OTHER_perc,SOLAR_perc,STORAGE_perc,GENERATION_perc,LOW_CARBON_perc,ZERO_CARBON_perc,RENEWABLE_perc,FOSSIL_perc
0,2009-01-01 00:00:00+00:00,8377.0,15037.0,7099.0,292.0,246,2516.0,0.0,0.0,0.0,...,7.5,0.0,0.0,0.0,0.0,100.0,22.8,22.8,1.6,69.8
1,2009-01-01 00:30:00+00:00,8508.0,15095.0,7087.0,269.0,245,2493.0,0.0,0.0,0.0,...,7.4,0.0,0.0,0.0,0.0,100.0,22.6,22.6,1.5,70.0
2,2009-01-01 01:00:00+00:00,8481.0,15086.0,7074.0,243.0,246,2462.0,0.0,0.0,0.0,...,7.3,0.0,0.0,0.0,0.0,100.0,22.5,22.5,1.5,70.2
3,2009-01-01 01:30:00+00:00,8326.0,15028.0,7064.0,225.0,246,2435.0,0.0,0.0,0.0,...,7.3,0.0,0.0,0.0,0.0,100.0,22.6,22.6,1.4,70.1
4,2009-01-01 02:00:00+00:00,8301.0,14995.0,7052.0,207.0,246,2361.0,0.0,0.0,0.0,...,7.1,0.0,0.0,0.0,0.0,100.0,22.6,22.6,1.4,70.2


In [4]:
# Extract the year and month for grouping
df["YEAR_MONTH"] = df["DATETIME"].dt.to_period("M")

# Group by the year and month and sum up the columns
monthly_data = df.groupby("YEAR_MONTH").sum(numeric_only=True).reset_index()

# Rename columns and format the output
monthly_data["DATE"] = monthly_data["YEAR_MONTH"].dt.start_time.dt.strftime("%Y-%m-%dT00:00:00.000")
monthly_data = monthly_data.drop(columns=["YEAR_MONTH"])

# Select relevant columns
columns = ['DATE', 'GAS', 'COAL', 'IMPORTS', 'STORAGE', 'OTHER', 'NUCLEAR', 'WIND', 'HYDRO', 'BIOMASS', 'SOLAR', 'GENERATION']

monthly_data = monthly_data[columns]

# Divide all sums by half (because all half-hourly records were measured in Megawatt hours)
for column in columns:
    if column != 'DATE':
        monthly_data[column] = monthly_data[column] / 2

# Remove current month's data
monthly_data = monthly_data.iloc[:-1]

monthly_data.head()

C:\Users\SamJansen\AppData\Local\Temp\ipykernel_1860\3080379824.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df["YEAR_MONTH"] = df["DATETIME"].dt.to_period("M")


,DATE,GAS,COAL,IMPORTS,STORAGE,OTHER,NUCLEAR,WIND,HYDRO,BIOMASS,SOLAR,GENERATION
0,2009-01-01T00:00:00.000,11382843.5,15424234.5,441428.0,341107.0,0.0,4311609.0,431297.0,405521.0,0.0,0.0,32738038.0
1,2009-02-01T00:00:00.000,9825972.0,12609886.5,416291.0,337643.0,0.0,5418402.5,282460.0,271427.5,0.0,0.0,29162074.5
2,2009-03-01T00:00:00.000,11445751.0,9968234.0,513558.5,313279.0,0.0,6342280.5,477814.5,430972.5,0.0,0.0,29491880.0
3,2009-04-01T00:00:00.000,11082214.0,6647467.0,948926.5,219944.0,0.0,6037318.5,317024.0,279276.0,0.0,0.0,25532168.0
4,2009-05-01T00:00:00.000,11589628.5,6309402.5,1299879.5,286622.0,0.0,5079225.0,406321.5,229834.0,0.0,0.0,25200898.5


In [5]:
# Create the list of dictionaries in the required format
result = monthly_data.to_dict(orient="records")

# Convert to JSON format
json_output = json.dumps(result, indent=2)

# Save to a file
with open("gen_mix_clean.json", "w") as f:
    f.write(json_output)

# Print the JSON output
print(json_output)

[
  {
    "DATE": "2009-01-01T00:00:00.000",
    "GAS": 11382843.5,
    "COAL": 15424234.5,
    "IMPORTS": 441428.0,
    "STORAGE": 341107.0,
    "OTHER": 0.0,
    "NUCLEAR": 4311609.0,
    "WIND": 431297.0,
    "HYDRO": 405521.0,
    "BIOMASS": 0.0,
    "SOLAR": 0.0,
    "GENERATION": 32738038.0
  },
  {
    "DATE": "2009-02-01T00:00:00.000",
    "GAS": 9825972.0,
    "COAL": 12609886.5,
    "IMPORTS": 416291.0,
    "STORAGE": 337643.0,
    "OTHER": 0.0,
    "NUCLEAR": 5418402.5,
    "WIND": 282460.0,
    "HYDRO": 271427.5,
    "BIOMASS": 0.0,
    "SOLAR": 0.0,
    "GENERATION": 29162074.5
  },
  {
    "DATE": "2009-03-01T00:00:00.000",
    "GAS": 11445751.0,
    "COAL": 9968234.0,
    "IMPORTS": 513558.5,
    "STORAGE": 313279.0,
    "OTHER": 0.0,
    "NUCLEAR": 6342280.5,
    "WIND": 477814.5,
    "HYDRO": 430972.5,
    "BIOMASS": 0.0,
    "SOLAR": 0.0,
    "GENERATION": 29491880.0
  },
  {
    "DATE": "2009-04-01T00:00:00.000",
    "GAS": 11082214.0,
    "COAL": 6647467.0,
    "IMP